# Efficient Pourbaix diagrams of high-dimensional composition spaces

*Joseph Montoya, Anjli Patel, Jens K. Nørskov, Kristin Persson*

Pourbaix diagrams are a valuable tool for exploring the comparative thermodynamic stability of solids and solution-phase ionic species as a function of ambient pH and electrochemical potential.  In addition, high-throughput computational materials science efforts like those from the Materials Project have enabled Pourbaix diagrams to be constructed and disseminated from computational data. However, Pourbaix analysis of phase stability on these resources have been limited to 3 or fewer elements, largely because higher compositions have proven extremely inefficient with existing methods and software infrastructure.  In this report, we identify details of a modified algorithm for the Pourbaix diagram construction which enables diagrams to be constructed efficiently in much higher compositional spaces, which enables phase stability analysis of similarly complex individual materials.  We demonstrate this functionality with an analysis of the phase stability of a few compounds with many elements in their composition and highlight our improved online implementation on the Materials Project website.

The primary bottleneck in current implementations of Pourbaix diagram software is in their iteration over potential combinations of compounds in preprocessing steps related to the enumeration of all possible entries to include in the Pourbaix analysis.  This means that the scaling of Pourbaix diagram construction occurs with n choose m, effectively O(n!) where n is the number of entries and m is the number of elements included.  Since larger numbers of elements tend to produce more entries on queries of the database, Pourbaix diagrams become prohibitively expensive after 3 elements.  More explicitly 4, 5, and 6 element Pourbaix diagrams for the Ba-Ca-V-Cu, Ba-Ca-V-Cu-F, and Ba-Ca-V-Cu-F-Fe chemical systems have estimated completion times of 60 hours, 50 years, and 1 million years, respectively.  Considerable speedup is achieved by filtering for entries on the convex hull of the solid phase diagram, which is at least partially motivated by physical reasoning that those materials should appear in the Pourbaix diagram absent any ions.  This process, however, is complicated by the variable chemical potential of H and O on the Pourbaix diagram (but not on the compositional phase diagram) and the need to add ionic species, which still results in poor combinatoric scaling. The process was also further improved (e. g. in pymatgen) by virtue of it being easily parallelized, but this still only renders a factor of N speedup when much larger factors are required for the higher-element spaces.

To circumvent this obstacle, one can compute the convex hull in a similar manner as a pymatgen-implemented compositional phase diagram, but in npH-nphi-E0-x1-x2…xn-1 space, which scales linearly with the number of points, rather than combinatorially.  This is effective because this convex hull is isomorphic to that in xe--xH+-x1-x2…xn-1+Hf0 space, which is effectively a compositional phase diagram that includes electrons and solution-phase protons at standard conditions in addition to the constituent elements of the included materials.  By excluding the end points end-points of the electron and proton species’ formation energies, all stable species in Pourbaix space are found on this convex hull.  These include entries with positive formation energies at standard conditions which would eventually become stable upon the increase of the chemical potential of either protons or electrons.  A schematic of the convex hull in this phase-space is shown in Figure 2 for the Zn and the Al-Ti chemical systems.

Upon yielding this convex hull, our new implementation still performs an iterative procedure on the facets to determine whether or not a valid combination of entries at their vertices can be determined.  However, by filtering only for facets which could reasonably have an interior point in composition space, and by virtue of the fact that there are far fewer combinations among the convex hull facets than in the general pool of candidate entries, many orders of magnitude of speedup are achieved.  For example, the compound on the Materials Project with the largest number of elements (mp-id, chemical system) can have its Pourbaix phase stability mapped in 5 minutes seconds.

We envision that this functionality will have particular applications in the evaluation of corrosion resistance of complex alloys and of the stability of catalysts in high-throughput studies of water splitting and fuel cell reactions.  We demonstrate four such examples in Figure 4. As such, we have disseminated the implementation in the pymatgen open-source software, enabling its use on the Materials Project website.  We note also that the pourbaix diagram infrastructure is agnostic to the determination of Gibbs free energy for a particular material, and therefore can include entropic effects which may govern the thermodynamics of phase behavior in materials with larger numbers of elements, e. g. high-entropy alloys.  Ultimately, it is our hope that efficient Pourbaix analysis of these complex compounds will enable new insights to be derived in these fields.

### Figure 1: Benchmarking performance

### Figure 2: A compositional phase diagram with electrons and protons

### Figure 3: Example pourbaix diagrams